
---

<big><big><big><big><big><big><big><big><big>Sieci neuronowe</big></big></big></big></big></big></big></big></big>

---


---

<id=tocheading><big><big><big><big><big>Spis treści</big></big></big></big></big>
<div id="toc"></div>

---

In [9]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
sns.set(font_scale=2.5)

In [3]:
# -*- coding: utf-8 -*-
"""
Demo of unicode support in text and labels.
"""
from __future__ import unicode_literals
from __future__ import print_function
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.datasets import load_svmlight_file, load_boston, load_breast_cancer
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, matthews_corrcoef, confusion_matrix
from sklearn import preprocessing
from sklearn.datasets import load_boston, load_diabetes, load_linnerud, make_regression
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3, make_sparse_uncorrelated
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [4]:
from astroML.datasets import fetch_rrlyrae_combined
from astroML.utils import split_samples, completeness_contamination

# Ograniczenia modeli
* modele liniowe mają ograniczenia z powodu klątwy wymiarowości
* niewystarczające są __ustalone__ funkcje bazowe
  * zły wybór może powodować overfitting  
  * trzeba __adaptować__ funkcje do konkretnych danych
  * ograniczyć liczbę funkcji bazowych

* alternatywne podejście
  * ustalić funkcje bazowe z góry - niewielką liczbę, zwykle __jedną__
  * niech funkcje __adaptują__ się w trakcie nauki
  * model jest zwykle znacznie mniejszy
  * funkcja ___likelihood_ nie będzie już wypukła__
    * konieczność przeglądnięcia większej liczby modeli

# Neuron biologiczny

  <img src="../mum_figures/neuron.png" width="100%"/>
  
  <img src="../mum_figures/synapse.jpg" width="100%"/>

![Potential](file:///Users/igorpodolak/Dropbox/mum_figures/potential.gif)

![Awesome cat gif](http://media.giphy.com/media/7z2oyDXIMEs8w/giphy.gif)

![Potential](http://github.com/gmum/ml2017/wyklady/mum_figures/potential.gif)

<img src="../mum_figures/potential.jpg" width="100%"/>

# Sieci z propagacją sygnału (feed-forward)
<img src="../mum_figures/mlp.png" width="100%"/>
* funkcje bazowe modelu
  * muszą __zależeć__ od parametrów
  * parametry powinny się zmieniać w trakcie uczenia
  * współczynniki $w_j$ także są modyfikowalne
* $M$ liniowych kombinacji wejść $$v_j=\sum_i^Dw_{ji}x_i+w_{j0}$$
gdzie $M$ jest liczbą __"neuronów"__ w pierwszej __warstwie__ modelu
  * $w_{j0}$ to __bias__ $j$-tego neuronu
  * $v_j$ to __aktywacje__
  * teraz użyta jest __funkcja aktywacji__ $\varphi()$ (różniczkowalna) $$z_j=\varphi(v_j)$$
    * funkcje aktywacji są zwykle funkcjami sigmoidalnymi, np.
    $$\frac{1}{1+\exp({-v})}, \tanh(-v)$$
  * neurony w warstwach, które nie są ani wejściową ani ukrytą nazywamy __ukrytymi__
  
  
* obliczenia powtarzają się w kolejnych warstwach
* końcowa wartość jest jako, dla sieci z jedną warstwą ukrytą
$$y_k(x)=\sigma\left(\sum_k^Mw_kj\sigma\left(\sum_i^Dw_{ji}x_i+w_{j0}\right)+w_{k0}\right)$$
  * jeśli problem regresji, to końcowa funkcja aktywacji jest identycznością
  * sieć neuronowa jest __nieliniową__ funkcją wejść $x_i$
    * wartość bias włączamy do wektora $w$ __rozszerzając__ wejście o $x_0=1$

* sygnał jest przesyłany do przodu, stąd nazwa
* sieć wielowarstwowa nazywana __wielowarstwowym perceptronem__
  * zaproponowany przez McCullocha i Pittsa __perceptron__ 
    * wykorzystywał __progową__ funkcję aktywacji (f. Heaviside'a)
    * nie zawierał warstw ukrytych

* sieć z __liniowymi__ f. aktywacji w w. ukrytych jest redukowalna do sieci __bez__ warstw ukrytych
* sieci warstwowe można uogólnić przez dodanie __połączeń skrótowych__

* nazewnictwo
  * istnieje wiele zwyczajów: sieć na rysunku może być nazywana 
    * 4-warstwową (bo liczba wszystkich warstw neuronów), 
    * 3-warstwową (bo liczba warstw adaptowalnych parametrów), 
    * 2-warstwową (bo liczba warstw ukrytych)
  * Bishop preferuje drugie podejście
  * jasne będzie "sieć z dwoma warstwami ukrytymi"

# Sieci neuronowe jako uniwersalny aproksymator
> Sieć neuronowa z jedną warstwą ukrytą neuronów z nieliniowymi ciągłymi monotonicznie rosnącymi funkcjami aktywacji może dowolnie dokładnie dla $\epsilon>0$ aproksymować dowolną funkcję ciągła zdefiniowaną na hiperkostce $[0, 1]^m$.

> Twierdzenie jest o _istnieniu_ takiej sieci, a nie mówi jak znaleźć odpowiednią liczbę neuronów.

## Funkcja kosztu: regresja
* niech przewidywana pojedyncza zmienna $t$ ma rozkład normalny zależny od wejściowego $x$ $$p(t\mid x)=\mathcal{N}(t\mid y(x, w), \sigma)$$ gdzie
  * $y(x, w)$ jest wyjściem sieci
  * $\sigma$ jest wariancją
* $X=\{x_1,x_2,\dots,x_N\}$ zbiorem $N$ __niezależnych__ przykładów
* dla wartości spodziewanych (target) $\{t_1,\dots,t_N\}$ można utworzyć __likelihood__ 
$$p(t\mid X,w,\sigma)=\prod_{n=1}^Np(t_n\mid x_n,w,\sigma)$$

* biorąc __ujemny logarytm__ dostajemy __log-likelihood__
$$\frac{1}{2\sigma}\sum_{n=1}^N\left(y(x_n,w)-t_n\right)^2+\frac{N}{2}\ln\sigma +\frac{N}{2}\ln(2\pi)$$
* maksymalizacja likelihood jest równoważna __minimalizacji__ log-likelihood, co jest identyczne z minimalizacją funkcji kosztu
$$E(w)=\frac{1}{2}\sum_{n=1}^N\left(y(x_n,w)-t_n\right)^2$$
  * nieliniowość funkcji $y(x,w)$ powoduje, że $E(w)$ __nie jest wypukła__
  * funkcja kosztu będzie miała __minima lokalne__
  * po znalezieniu wektora $w^\ast$ można znaleźć $\sigma$ jako
  $$\sigma=\frac{1}{N}\sum_{n=1}^N\left(y(x_n,w^\ast)-t_n\right)^2$$



## Funkcja kosztu: klasyfikacja
* przypadek klasyfikacji binarnej
  * pojedynczy neuron wyjsciowy z $\sigma()$ __logistyczną__
  $$y=\sigma(v)=\frac{1}{1+\exp(-v)}$$
  * niech $t=1$ jeśli $x\in C_1$; $t=2$ jeśli $x\in C_2$
  * niech $y(x,w)$ będzie __prawdopodobieństwem__, że $x\in C_1$
  * $1-y(x,w)$ prawdopodobieństwem, że $x\in C_2$
  
* rozkład warunkowy wyjść rozkładem Bernoulliego
$$p(t\mid x)=y(x,w)^t\left(1-y(x,w)\right)^{1-t}$$

* dla zbioru wszystkich przykładów $X$ mamy likelihood
$$p(t\mid X,w)=\prod_ny(x_n,w)^t_n\left(1-y(x_n,w)\right)^{1-t_n}$$
* znowu ujemny logarytm daje log-likelihood i funkcję kosztu
$$E(w)=-\sum_{n=1}^N\left[t_n\ln\,y_n+(1-t_n)\ln\,(1-y_n)\right]$$
nazywaną __entropią krzyżową__
  
  * dla klasyfikacji użycie entropii krzyżowej daje lepsze wyniki niż użycie funkcji kwadratowej kosztu
* dla problemu wieloklasowego $t\in\{C_1,\dots,C_K\}$ mamy
$$E(w)=-\sum_{n=1}^N\sum_{k=1}^K\left[t_{nk}\ln\,y_k(x_n,w)+(1-t_{nk})\ln\,(1-y_k(x_n,w))\right]$$
* dla klasyfikacji wieloklasowej mamy
  $$y_k(x,w)=p(t_k=1\mid x)$$
  * to skraca wyrażenie entropii krzyżowej do
  $$E(w)=-\sum_{n=1}^N\sum_{k=1}^Kt_n\ln\,y_k(x_n,w)$$
  * dla problemu wieloklasowego aktywacją wyjściową jest funkcja softmax
  $$y_k(x,w)=\frac{\exp(v_k(x,w))}{\sum_{j=1}^K\exp(v_j(x,w))}$$

## Cechy
* podstawowa różnica do modeli liniowych: sieć neuronowa __mapuje przestrzeń wejściową do przestrzeni ukrytej gdzie znajduje rozwiązanie__
  * __nieliniowe__ rzutowanie do przestrzeni ukrytej zwiększa prawdopodobieństwa liniowej separowalności

* aktywacje warstw ukrytych dla danego wejścia nazywamy cechami
  * w modelach każda przynależność do klasy jest rozwiązywana jako __całkowicie osobny__ problem
  * w sieciach podproblemy przynależności do klas __dzielą cechy__

# Uczenie sieci warstwowych neuronowych
* zadaniem jest minimalizacja $E(w)$
* drobna zmiana $w$ powoduje __presunięcie__ się po powierzchni funkcji kosztu
  * dualność: każde rozwiązanie odpowiada punktowi w przestrzeni wag
* przestrzeń rozwiązań jest zwykle bardzo wysoko wymiarowa
  * istnieje wiele punktów o $\nabla{}E(w)\simeq0$
  * to minima/maksima lokalne/globalne, punkty siodłowe

## Lokalna aproksymacja kwadratowa
* rozwinięcie funkcji kosztu w szereg Taylora
$$E(w)\simeq E(w_0)+(w-w_0)g+\frac{1}{2}(w-w_0)H(w-w_0)$$
gdzie 
$$\begin{align}
g=&\frac{\partial E}{\partial w}\\
H=&\frac{\partial^2 E}{\partial w_i\,\partial w_j}
\end{align}$$

* stąd
$$\nabla E\simeq g+H(w-w_0)$$

## Uczenie gradientowe
* uczenie małymi krokami
$$w(t+1)=w(t)-\eta\nabla E(w(t))$$
gdzie $\eta$ jest __współczynnikiem nauczania__

## Tryby uczenia gradientowego

* to metoda __epoka po epoce__ (ang. batch)
  * każdy krok poprawy wymaga wielu obliczeń
  * każda propagacja wprzód używa starych wag
  * wiele przykładów dzieli wspólne cechy, stąd utrata wielu możliwych informacji
  * uczenie po epokach może powodować krążenie w przestrzeni rozwiązań
  * usprawnienie: metoda gradientów sprzężonych zapewniająca poprawy
    * każda poprawa jest ortogonalna do poprzednich
    * teoretycznie roziązanie po co najwyżej tylu krokach ile jest wag

* epoka __przykład po przykładzie__ (ang. stochastic gradient descent)
$$E(w)=\sum_nE_n(w)$$
  * poprawa po każdym przykładzie
  $$w(t+1)=w(t)-\eta\nabla E_n(w(t))$$
  * uwzględnia wiedzę z innych przykładów od razu
  * pozwala na łatwiejsze omijania minimów lokalnych
 
* __mini batch__
  * uczenie małymi grupami przykładów
  * najbardziej efektywne

# Wsteczna propagacja błędów
* sieci przechodziły przez wzloty i upadki
* zapowiadano (Minsky i Papert w _Perceptron_) śmierć ze względu na __niemożność__ uczenia sieci z warstwami ukrytymi
* problem z liczeniem gradientu dla warstw ukrytych
* algorytm wstecznej propagacji zaproponowany przez Werbosa


* w trakcie uczenia obliczana jest duża liczba pochodnych funkcji kosztu __względem__ wag
* wsteczna propagacja jest __efektywnym__ machenizmem ich obliczania
  * pochodne (błędy) są przesyłane wstecz w sieci
* drugim elementem jest obliczenie i modyfikacja wartości wag

* schemat może być użyty do szerokiego wachlarza architektur sieci



Funkcja kosztu $$E(w)=\sum_{n=1}^NE_n(w)$$
<img src="../mum_figures/backpropagation.png" width="70%"/>
1. neurony wyjściowe w modelu __liniowym__
  * $y_k=\sum_kw_{ki}x_i$ i funkcją kosztu 
  $$E_n=\frac{1}{2}\sum_k(y_{nk}-t_{nk})^2$$
  * uczenie polega na __modyfikacji__ wag w celu __minimalizacji__ kosztu
  
  $$\frac{\partial E_n}{\partial w_{kj}}=(y_{nj}-t_{nj})x_i$$
  co jest uczeniem __przez poprawianie błędu__
2. modele wielowarstwowe
  * aktywacja neuronu w warstwie to
  
  $$z_j=\varphi(v_j)=\varphi\left(\sum_iw_{ji}z_i\right)$$
  z nieliniową funkcją aktywacji $\varphi(\cdot)$
  * koszt jest zależny od wag __tylko__ przez sumę ważoną $v_j$
  
  $$\frac{\partial E_n}{\partial w_{ji}}=\frac{\partial E_n}{\partial v_j}\frac{\partial v_j}{\partial w_{ji}}$$
  <img src="../mum_figures/backpropagation.png" width="80%"/>
  * wprowadzamy oznaczenie __delta__
  
  $$\delta_j\equiv\frac{\partial E_n}{\partial v_j}$$
  odpowiadające lokalnemu wpływowi na całkowity błąd
  
  $$\begin{align}
  \frac{\partial E_n}{\partial w_{ji}}&=\frac{\partial E_n}{\partial v_j}\frac{\partial v_j}{\partial w_{ji}}\\
  &=\delta_j\frac{\partial v_j}{\partial w_{ji}}\\
  &=\delta_jz_i
  \end{align}$$
    * dla neuronów (liniowych) __wyjściowych__ mamy
    
    $$\delta_k=\frac{\partial E_n}{\partial v_k}=y_k-t_k$$
    * dla neuronów __ukrytych__ (tu w warstwie __poprzedzającej__ wyjściową)
    
    $$\begin{align}
    \delta_j&=\frac{\partial E_n}{\partial v_j}\\
    &=\sum_k\frac{\partial E_n}{\partial v_k}\frac{\partial v_k}{\partial v_j}\\
    &=\sum_k\delta_k\frac{\partial v_k}{\partial v_j}\\    &\hskip{4em}\text{gdzie}\hskip{1em}v_k=\sum_jw_{kj}z_j=\sum_jw_{kj}\varphi(v_j)\\
    &=\varphi'(v_j)\sum_kw_{kj}\delta_k
    \end{align}$$
    <img src="../mum_figures/backpropagation.png" width="100%"/>
      * wartości $\delta_k$ zostały już obliczone dla wartwy __późniejszej__ w modelu sieci
      * $\delta_j$ odpowiadają __wpływowi $j$-tego neuronu na błędy w późniejszych warstwach__, czyli __loklanemu błędowi $j$-tego ukrytego neuronu__
      * stąd __wsteczna propagacja błędów__

### Funkcje aktywacji
* zwykle __sigmoidalne__
  * wprowadzają __nieliniowość__
  * __logistyczna__ $$\sigma(v)=\frac{1}{1+exp(-v)}$$
    * pochodna prosta obliczeniowo $$\sigma'(v)=\sigma(v)(1-\sigma(v))$$
  * __tangens hiperboliczny__
    * aktywacja na $(-1, 1)$
    * preferowany dla niskich warstw
      * pozwala by wymusić średnią aktywacji $0$ co zwykle przyspiesza uczenie
  * gradient dla wysokich wartości bezwzględnych __zanika__
    * tzw. __vanishing gradient problem__
    * uczenie zanika mimo błędu
* problem zanikającego gradientu utrudnia uczenie sieci o wielu warstwach ukrytych
  * propozycja aktywacji __rectified linear__ $$\varphi(v)=\max(0, v)$$
    * gradient nie zanika
    * możliwe modyfikacje

### Co się dzieje w warstwie ukrytej (3 różne klasy)

* aktywacje oczekiwne
<img src="../mum_figures/cl3-in.png" width="50%"/>
* aktywacje uzyskane
<img src="../mum_figures/cl3-out.png" width="50%"/>
* aktywacje warstwy ukrytej (kolory według klasy uzyskanej)
<img src="../mum_figures/cl3-hid.png" width="50%"/>
* błąd aktywacji
<img src="../mum_figures/cl3-err.png" width="50%"/>

### Co się dzieje w warstwie ukrytej (4 różne klasy)

* aktywacje oczekiwne
<img src="../mum_figures/cl4-in.png" width="50%"/>
* aktywacje uzyskane
<img src="../mum_figures/cl4-out.png" width="50%"/>
* aktywacje warstwy ukrytej (kolory według klasy uzyskanej)
<img src="../mum_figures/cl4-hid.png" width="50%"/>
* błąd aktywacji
<img src="../mum_figures/cl4-err.png" width="50%"/>

## Czy wsteczna propagacja jest efektywna obliczeniowo?
* liczba obliczeń przy obliczaniu aktywacji jest proporcjonalna do liczby wag
  * wag jest __znacznie__ więcej niż neuronów
* obliczenia dla funkcji kosztu też są rzędu liczby wag

* numeryczne obliczanie pochodnych 

$$\frac{\partial E}{\partial w}=\frac{E(w+\epsilon)-E(w)}{\epsilon}+O(\epsilon)$$
lub

$$\frac{\partial E}{\partial w}=\frac{E(w+\epsilon)-E(w-\epsilon)}{2\epsilon}+O(\epsilon^2)$$
ma wyższą złożoność $O(w^2)$

## Obliczenia drugiego rządu
* macierz drugich pochodnych

$$\frac{\partial^2 E}{\partial w_j \partial w_k}$$
* macierz Hesjanu $$H=\nabla\nabla E$$
  * pozwala w optymalizacju brać pod uwagę __krzywiznę__ powierzchni błędu
    * gradient tylko kierunek zmian
  * odwrotność H umożliwia znalezienie nieistotnych wag w sieci (pruning)
  * szybkie douczanie po zmianie przykładów
    * niewielka zmiana położenia minimum
* wyższa złożoność $O(W^2)$

# Regularyzacja
* zwykle bardzo duża liczba parametrów (wag)
* konieczność długich testów dla znalezienia optymalnych parametrów
  * liczba neuronów, wartości wag, etc.

1. __weight decay__ (__L2__)

$$\overline{E}(w)=E(w)+\frac{1}{2}w^Tw$$

2. __L1__
$$\overline{E}(w)=E(w)+\|w\|_1$$

2. problemy z liniową transformację
  * skalowanie wejść jest __niespójne__ z prostą regularyzacją __ weight decay__
    * translacja (skalowanie) wejść (lub wyjść) da nierównoważną (przez liniowe przekształcenie) sieć przy uczeniu z weight decay
    * translacje inaczej wpływają na różne warstwy sieci
  * osobny czynnik egularyzacyjny dla osobnych warstw sieci
  
  $$\frac{\lambda_1}{2}\sum_{w\in W_1}w^2+\frac{\lambda_2}{2}\sum_{w\in W_2}w^2$$
  
3. wczesne zatrzymywanie
  * błąd na zbiorze uczącym maleje w trakcie uczenia
  * błąd na zbiorze __walidującym__ zaczyna rosnąć
    * jest to znak rozpoczęcia zbytniego dopasowania
  <img src="../mum_figures/generalization.png" width="120%"/>



## Alternatywy
* wsteczna propagacja _per se_ jest powolna
* blisko minimum może wpaść w oscylacje
* problemy przy niskim spadku
* metoda __momentum__

$$w(t+1)=w(t)-\eta\nabla E(t) -\alpha\Delta(w(t-1))$$
  * szereg usprawnień, np. algorytm Adam
* metody adaptacynego kroku
  * atrybuty wejsciowe nie mają równego wpływu na rozwiązanie
  * osobne parametry uczenia dla każdego z nich
  * algorytm delta-delta
* problemy z _płaską_ powierzchnią błędu
  * metoda __resilient backpropagation__
  
  $$\Delta w_{ji}(t+1)=\begin{cases}
  -\eta_{ji}sgn(\nabla_{ji} E(t))&\text{jeśli}\;\;\nabla_{ji}E(t)\nabla_{ji}E(t-1)\geq0\\
  0; \nabla_{ji}E(t)=0&\text{w przeciwnym przypadku}
  \end{cases}$$
  * parametr też potrzebuje być modyfikowany
  $$\eta_{ji}(t+1)=\begin{cases}
  min(u*\eta_{ji}, \eta_{max})&\nabla_{ji}E(t)\nabla_{ji}E(t-1)>0\\
  max(d*\eta_{ji}, \eta_{min})&\nabla_{ji}E(t)\nabla_{ji}E(t-1)<0\\
  \eta_{ji}&\text{wpp}
  \end{cases}$$
  dla $u>1$ i $d<1$, np. $u=1.05$ i $d=0.7$
    * podwyższany jeśli kolejne gradienty w tym samym kierunku
    * zmniejszany jeśli w przeciwnym
* bardzo efektywne podejście dla algorytmów klasyfikacji

* wiele obecnych algorytmów, np. Adam, współdzielą wiele cech

# Sieci konwolucyjne
* wiele cech jest dzielonych
* mogą wystąpić w __różnych__ częściach wejścia (obrazy)
* konieczna niezależność (chociaż nawet niewielka) na translację/skalowanie
  1. utworzyć bardzo duży zbiór uczący wszelkich możliwości
  
  
  
* lokalne cechy
  * niewielkie obszary wejściowe
  * filtry/konwolucje
  * struktura kratki
    * jedno-wymiarowe szeregi czasowe
    * dwu-wymiarowe obrazy
    * czasem trój-wymiarowa (obrazy medyczne)
* dzielenie wag
  * te same cechy w różnych obszarach
  * ograniczenie liczby rzeczywistych parametrów
* subsampling
  * odporność na translacje
* hierarchiczność
  * wykrywanie prostych cech i ich składanie
  
  
* model Neocognitron Fukushimy (1980)
  <img src="../mum_figures/neocognitron.png" width="120%"/>



### AlexNet (ImageNet, KSH) (Krizhevsky, Sutskeyer, Hinton)
<img src="../mum_figures/alexnet.jpg" width="120%"/>

<img src="../mum_figures/conv-images.png" width="120%"/>

## Konwolucje
1. jedno-wymiarowe
  * pomiar danych w których ostatnie odczyty są bardziej znaczące (szeregi czasowe)
  
  $$s(t)=\int x(a)w(t-a)da$$
  jest __konwolucją__ gdzie $a$ jest wiekiem pomiaru, zwykle $s(t)=(x\ast w)(t)$
    * $x$ to __wejście__
    * $w$ __jądro__ (kernel)
    * wyjście jest __cechą__
  * pomiary są dyskretne
  
  $$s(t)=\sum_{a=-\infty}^\infty x(a)w(t-a)$$
    * wartości jądra są równe $0$ poza skończoną liczbą pozycji
2. dwu-wymiarowe
  * filtr na obrazie $I$
  
  $$\begin{align}
  s(i,j)&=(I\ast K)(i, j)=\sum_m\sum_n I(m,n)K(i-m,j-n)\\
  &=(K\ast I)(i, j)=\sum_m\sum_n I(i-m,j-n)K(m,n)
  \end{align}$$
  * zwykle implementowane jest __korelacja krzyżowa__, równoważna
  
  $$s(i,j)=(K\ast I)(i, j)=\sum_m\sum_n I(i+m,j+n)K(m,n)$$

### Cechy
* rzadkie połączenia, rzadkie wagi
  * kernele znacznie mniejsze od obrazu
    * efektywność obliczeniowa
  * pojedyncze wejścia wpływają tylko na niewiele wyjść
  * na pojedynczą konwoulcję (wyjście) wpływa niewiele wejść
* lokalność cech
  * konwolucje wyszukują lokalne cechy
  * pozwala na pozbycie się kontekstu
  <img src="../mum_figures/pixels_embed_cifar10.jpg" width="100%"/>
  
* dzielenie wag
  * tradycyjnie jedna waga między parą wejście-wyjście
  * konwolucje pozwalają na dzielenie wag
  * zmniejszenie liczby parametrów i wielkości pamięci
  * pozwala na uniezależnienie od translacji
    * te same cechy wykrywane w szeregach czasowych w różnych okresach
    * te same filtry aktywowane na obrazach
    
* konwolucje w niskich warstwach wykrywają zwykle brzegi, krawędzie
  * to pozwala na tzw. __transfer learning__
  
* __padding__ to operacja __poszerzania__ wejścia o zerowe wejścia na brzegu
  * możliwe na każdym poziomie
  * w przeciwnym wypadku wymiar zmniejsza się
    * dla wejścia o szerokości $m$ i jądrze o szerokości $k$ wyjście będzie miało $m-k+1$ pikseli
      * bardzo szybkie zmniejszanie dla duzych kerneli

## Pooling
* odporność na __lokalne translacje__
* wybór z ograniczonego obszaru najsilniejszej (max-pooling) aktywacji
  * odpowiada wybraniu cechy najsilniejszej w obszarze
  * istotne znalezienie cechy bez dokładnego określenia położenia
* pooling zwykle ogranicza wielkość obszaru
  * uogólnianie cech
  
* czasem proponowane jest opuszczenie warstw pooling
  * propozycje użycia konwolucji z większym przesunięciem (ang. stride)